#### This notebook is join the rejection production data with the weather data 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyspark
from IPython.display import display, HTML
from functools import reduce
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import concat, lit
from pyspark.sql.functions import desc,asc, monotonically_increasing_id, split, size, length

from pyspark.sql.types import StringType

In [3]:
%run /Users/p0z00cx/jupter/product_weather_prediction/lib/Feature_engineering_helpers.ipynb

In [4]:
%run /Users/p0z00cx/jupter/product_weather_prediction/lib/classifer_helpers.ipynb

In [5]:
%run /Users/p0z00cx/jupter/product_weather_prediction/lib/production_utility.ipynb

In [6]:
product_pd = spark.read.csv('/Users/p0z00cx/hactron_product/rejection_data_with_station_id.csv', header="true", inferSchema="true")
display(product_pd.toPandas())

,item_desc,dept_desc,vendor_name,inspected_result,order_date,country,dc_name,mabd_date,ordered_cases,state,region,station_id,start_day,end_day
0,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/4/19,United States of America (USA),"COLTON, CA FRESH",9/10/19,117,California,Watsonville,BR000970000,20190821,20190910
1,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,rejected,9/3/19,United States of America (USA),"COLTON, CA FRESH",9/9/19,74,California,Watsonville,BR000970000,20190820,20190909
2,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,aminus,9/3/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/11/19,117,California,Watsonville,BR000970000,20190822,20190911
3,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/2/19,United States of America (USA),"OCHELATA, OK GDC",9/8/19,234,California,Watsonville,BR000970000,20190819,20190908
4,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,9/6/19,United States of America (USA),"CLARKSVILLE, AR GDC",9/14/19,351,California,Watsonville,BR000970000,20190825,20190914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26165,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/5/20,Mexico,"LEWISTON, ME GDC",4/13/20,108,Jalisco,Mexico,MXN00014179,20200324,20200413
26166,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"BEDFORD, PA GDC",4/24/20,108,Jalisco,Mexico,MXN00014179,20200404,20200424
26167,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/16/20,Mexico,"JOHNSTOWN, NY GDC",4/23/20,108,Jalisco,Mexico,MXN00014179,20200403,20200423
26168,ORG BLKBERRY 6OZ,ORG MIXED BERRIES,DOLE FRESH VEGETABLES,accepted,4/12/20,Mexico,"JOHNSTOWN, NY GDC",4/19/20,216,Jalisco,Mexico,MXN00014179,20200330,20200419


In [7]:
weather_data_2020 = spark.read.csv('/Users/p0z00cx/Downloads/2020.csv', header="true", inferSchema="true")
weather_data_2019= spark.read.csv('/Users/p0z00cx/Downloads/2019.csv.gz', header="true", inferSchema="true")
weather_data = weather_data_2019.unionAll(weather_data_2020)
weather_data.count()

44457086

In [8]:
col_list=['stationId', 'date', 'type', 'value', 'item1', 'item2','item3','item4']
oldColumns = weather_data.schema.names
newColumns = col_list
weather_data = reduce(lambda weather_data, idx: weather_data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), weather_data)
weather_data.show(10)

+-----------+--------+----+-----+-----+-----+-----+-----+
|  stationId|    date|type|value|item1|item2|item3|item4|
+-----------+--------+----+-----+-----+-----+-----+-----+
|US1FLSL0019|20190101|SNOW|    0| null| null|    N| null|
|US1NVNY0012|20190101|PRCP|    0| null| null|    N| null|
|US1NVNY0012|20190101|SNOW|    0| null| null|    N| null|
|US1ILWM0012|20190101|PRCP|  163| null| null|    N| null|
|USC00141761|20190101|TMAX|  -11| null| null|    7|  700|
|USC00141761|20190101|TMIN| -139| null| null|    7|  700|
|USC00141761|20190101|TOBS| -133| null| null|    7|  700|
|USC00141761|20190101|PRCP|    0| null| null|    7|  700|
|USC00141761|20190101|SNOW|    0| null| null|    7| null|
|USC00141761|20190101|SNWD|    0| null| null|    7|  700|
+-----------+--------+----+-----+-----+-----+-----+-----+
only showing top 10 rows



#### Get the PRCP

In [9]:
join_condition = [product_pd.station_id == weather_data.stationId,
product_pd.start_day < weather_data.date,
product_pd.end_day > weather_data.date,
weather_data.type == "PRCP"
]

In [10]:
product_pd = product_pd.withColumn("id", monotonically_increasing_id())
product_pd.count()

26170

In [11]:
df3 = product_pd.join(weather_data, how='left', on=join_condition)
df3 = df3.filter("stationId is not null")
df3.count()

37579

In [12]:
df4 = df3.groupBy(['id']).agg(F.collect_list('value').alias('prcp_list'))
df4 = df4.withColumnRenamed("id", "id4")
display(df4)

DataFrame[id4: bigint, prcp_list: array<int>]

In [13]:
df4 = df4.withColumn("prcp_aver", get_average_udf("prcp_list"))

In [14]:
df4 = df4.drop("prcp_list")
df4.show(10)

+-----+------------------+
|  id4|         prcp_aver|
+-----+------------------+
| 8484|                 0|
|15437|               223|
|19907|                 0|
|21223|              11.5|
|15173| 53.78947368421053|
|15194|               337|
|15237|             237.5|
|15322|              67.3|
| 4823|                85|
|15371|104.52631578947368|
+-----+------------------+
only showing top 10 rows



In [15]:
product_pd_final = product_pd.join(df4, how="left", on=[product_pd.id == df4.id4])
product_pd_final = product_pd_final.drop("prcp_list").drop("id4")
product_pd_final.show(5)
product_pd_final.count()

+---------------+------------+--------------------+----------------+----------+--------------------+--------------------+---------+-------------+------------+-------------+-----------+---------+--------+----+---------+
|      item_desc|   dept_desc|         vendor_name|inspected_result|order_date|             country|             dc_name|mabd_date|ordered_cases|       state|       region| station_id|start_day| end_day|  id|prcp_aver|
+---------------+------------+--------------------+----------------+----------+--------------------+--------------------+---------+-------------+------------+-------------+-----------+---------+--------+----+---------+
|BLACKBERRY 12OZ|BLACKBERRIES|       DRISCOLLS INC|        rejected|   9/30/19|United States of ...|PAULS VALLEY, OK GDC|  10/8/19|          117|  California|  Watsonville|BR000970000| 20190918|20191008|  26|     null|
|BLACKBERRY 12OZ|BLACKBERRIES|       DRISCOLLS INC|        accepted|  10/18/19|United States of ...| CLARKSVILLE, AR GDC| 10

26170

#### get TAVG_list



In [16]:
join_condition = [product_pd.station_id == weather_data.stationId,
product_pd.start_day < weather_data.date,
product_pd.end_day > weather_data.date,
#weather_data.type == "TMIN" ,
#weather_data.type == "TMAX",
weather_data.type == "TAVG"
]

In [17]:
df3 = product_pd.join(weather_data, how='left', on=join_condition)
df3 = df3.filter("stationId is not null")
df3.count()
df3.show(3)

+------------------+---------+--------------------+----------------+----------+--------------------+---------------+---------+-------------+----------+------+-----------+---------+--------+-----+-----------+--------+----+-----+-----+-----+-----+-----+
|         item_desc|dept_desc|         vendor_name|inspected_result|order_date|             country|        dc_name|mabd_date|ordered_cases|     state|region| station_id|start_day| end_day|   id|  stationId|    date|type|value|item1|item2|item3|item4|
+------------------+---------+--------------------+----------------+----------+--------------------+---------------+---------+-------------+----------+------+-----------+---------+--------+-----+-----------+--------+----+-----+-----+-----+-----+-----+
|APPLE FUJI 3# 8 NY|   APPLES|NEW YORK APPLE SA...|        rejected|    4/4/19|United States of ...|ARCADIA, FL GDC|  4/14/19|          250|New Jersey|    NY|USR0000JEBF| 20190325|20190414|18021|USR0000JEBF|20190326|TAVG|   38| null| null|    U

In [18]:
df4 = df3.groupBy(['id']).agg(F.collect_list('value').alias('tavg_list'))
df4 = df4.withColumnRenamed("id", "id4")
df4.count()

3181

In [19]:
df4 = df4.withColumn("tavg", get_average_udf("tavg_list"))
product_pd_final = product_pd_final.join(df4, how="left", on=[product_pd_final.id == df4.id4])
product_pd_final = product_pd_final.drop("tavg_list").drop("id4")
product_pd_final.show(5)
product_pd_final.count()

+---------------+------------+--------------------+----------------+----------+--------------------+--------------------+---------+-------------+------------+-------------+-----------+---------+--------+----+---------+------------------+
|      item_desc|   dept_desc|         vendor_name|inspected_result|order_date|             country|             dc_name|mabd_date|ordered_cases|       state|       region| station_id|start_day| end_day|  id|prcp_aver|              tavg|
+---------------+------------+--------------------+----------------+----------+--------------------+--------------------+---------+-------------+------------+-------------+-----------+---------+--------+----+---------+------------------+
|BLACKBERRY 12OZ|BLACKBERRIES|       DRISCOLLS INC|        rejected|   9/30/19|United States of ...|PAULS VALLEY, OK GDC|  10/8/19|          117|  California|  Watsonville|BR000970000| 20190918|20191008|  26|     null|              null|
|BLACKBERRY 12OZ|BLACKBERRIES|       DRISCOLLS I

26170

In [20]:
product_pd_final = product_pd_final.fillna({'prcp_aver': '0'}).fillna({'tavg': '70'})
display(product_pd_final.toPandas())

,item_desc,dept_desc,vendor_name,inspected_result,order_date,country,dc_name,mabd_date,ordered_cases,state,region,station_id,start_day,end_day,id,prcp_aver,tavg
0,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,rejected,9/30/19,United States of America (USA),"PAULS VALLEY, OK GDC",10/8/19,117,California,Watsonville,BR000970000,20190918,20191008,26,0,70
1,BLACKBERRY 12OZ,BLACKBERRIES,DRISCOLLS INC,accepted,10/18/19,United States of America (USA),"CLARKSVILLE, AR GDC",10/25/19,117,California,Oxnard,BR000970000,20191005,20191025,29,0,70
2,STRAWBERRY 2#,STRAWBERRIES,BETTER PRODUCE INC,aminus,12/23/19,Mexico,"AUBURN, IN GDC",12/31/19,960,Michoacan,Zamora,MXN00014101,20191211,20191231,474,0,70
3,STRAWBERRY 2#,STRAWBERRIES,RED BLOSSOM SALES INC,accepted,4/6/20,United States of America (USA),"COLTON, CA FRESH",4/13/20,3200,California,Santa Maria,BR000970000,20200324,20200413,964,0,70
4,APPLE FUJI 3# 8,APPLES,RICE FRUIT COMPANY,accepted,12/29/19,United States of America (USA),"BEDFORD, PA GDC",1/4/20,300,Pennsylvania,South Central,USR0000PALL,20191215,20200104,1677,0,156.47368421052633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26165,AVO HASS 72 MX LGS,AVOCADOS,LGS SPECIALTY SALES LTD,accepted,3/17/20,Mexico,"BEDFORD, PA GDC",3/26/20,440,Michoacan,Uruapan,MXN00014101,20200306,20200326,25218,0,70
26166,STRAWBERRY 1# SB,STRAWBERRIES,SUPREME BERRY FARMS LLC,aminus,7/25/19,United States of America (USA),"NEW ALBANY, MS GDC",7/30/19,2160,None,None,None,20190710,20190730,25299,0,70
26167,STRAWBERRY 2# SB,STRAWBERRIES,SUPREME BERRY FARMS LLC,aminus,6/8/19,United States of America (USA),"MANKATO, MN GDC",6/13/19,3520,None,None,None,20190524,20190613,8589934668,0,70
26168,PMPKN BIN 40 IHP,PUMPKINS SEASONAL,INDIAN HILLS PRODUCE INC,aminus,10/12/19,United States of America (USA),"BRUNDIDGE, AL GDC",10/18/19,60,None,None,None,20190928,20191018,8589935178,0,70


In [21]:
test_data_set =  product_pd_final.select("item_desc", "dept_desc", "state","region","inspected_result","dc_name","prcp_aver", "tavg")
test_data_set = test_data_set.withColumn("result", udf(lambda x : 0 if x =="rejected" else 1)("inspected_result"))
display(test_data_set.toPandas())

,item_desc,dept_desc,state,region,inspected_result,dc_name,prcp_aver,tavg,result
0,BLACKBERRY 12OZ,BLACKBERRIES,California,Watsonville,rejected,"PAULS VALLEY, OK GDC",0,70,0
1,BLACKBERRY 12OZ,BLACKBERRIES,California,Oxnard,accepted,"CLARKSVILLE, AR GDC",0,70,1
2,STRAWBERRY 2#,STRAWBERRIES,Michoacan,Zamora,aminus,"AUBURN, IN GDC",0,70,1
3,STRAWBERRY 2#,STRAWBERRIES,California,Santa Maria,accepted,"COLTON, CA FRESH",0,70,1
4,APPLE FUJI 3# 8,APPLES,Pennsylvania,South Central,accepted,"BEDFORD, PA GDC",0,156.47368421052633,1
...,...,...,...,...,...,...,...,...,...
26165,AVO HASS 72 MX LGS,AVOCADOS,Michoacan,Uruapan,accepted,"BEDFORD, PA GDC",0,70,1
26166,STRAWBERRY 1# SB,STRAWBERRIES,None,None,aminus,"NEW ALBANY, MS GDC",0,70,1
26167,STRAWBERRY 2# SB,STRAWBERRIES,None,None,aminus,"MANKATO, MN GDC",0,70,1
26168,PMPKN BIN 40 IHP,PUMPKINS SEASONAL,None,None,aminus,"BRUNDIDGE, AL GDC",0,70,1


In [22]:
test_data_set.toPandas().to_csv('/Users/p0z00cx/hactron_product/test_data_set.csv', header="true", index=None)